<div dir='rtl'>
در این قسمت به سراغ استفاده از روش بردار پشتیبان می‌رویم.
<br>
با توجه یه اینکه ویژگی‌های داده بیشتر از نوع دسته‌بندی هستند، انتظار زیادی از این روش روی این داده نمی‌رود.
</div>

In [1]:
import pandas as ps
import numpy as np
import seaborn as sns
from sklearn import svm

import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/gdrive")


In [2]:

data = ps.read_csv('/content/gdrive/My Drive/train_data.csv' if IN_COLAB else 'train_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3768416 entries, 0 to 3768415
Data columns (total 16 columns):
 #   Column        Dtype
---  ------        -----
 0   displayId     int64
 1   timestamp     int64
 2   dayOfWeek     int64
 3   hourOfDay     int64
 4   advertiserId  int64
 5   campaignId    int64
 6   creativeId    int64
 7   publisher     int64
 8   widgetId      int64
 9   device        int64
 10  os            int64
 11  browser       int64
 12  source        int64
 13  docId         int64
 14  userId        int64
 15  clicked       int64
dtypes: int64(16)
memory usage: 460.0 MB


<div dir='rtl'>ابتدا به سراغ تقسیم‌بندی داده‌ها می‌رویم. با توجه به نامتوازن بودن داده‌ها، روش‌هایی مانند (undersampling) نمونه‌زایی (oversampling) و نمونه‌کاهی می‌تواند کمک کند اما به توجه به هزینه این روش‌ها که دور شدن از داده‌واقعی است، تلاش خود را بر اصلاح تابع خطا به شکلی که داده‌های کلاس نادر را مهمتر درنظر گیرد می‌گذاریم.
<br>
کتابخانه sklearn الگوریتم svm را به صورت وزن دار پیاده‌سازی کرده است. به طوری‌که میتوان با وزن‌دادن به کلاس‌ها، تشخیص درست‌ یک کلاس را بر دیگر کلاس‌ها اولویت داد. با توجه به این‌که تعداد نمایش‌های منجر به مراجعه، ۲۵ درصد کل داده است، وزن این کلاس راتعیین می‌کنیم.
<div>

In [3]:
clsf = svm.SVC(kernel='linear', C=1, class_weight={0:.25, 1:.75})

<div dir='rtl'>
اکنون عملیات روی داده‌ها را شروع می‌کنیم.
<br>
اول ویژگی‌های غیر ترتیبی را حذف می‌کنیم و بعد داده‌ی یادگیری را از داده‌ آزمون جدا می‌کنیم.
</div>

In [4]:
from sklearn.model_selection import train_test_split
data = data.sample(10000)

clean_data = data.drop(data.columns[[0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]] , axis=1)
print(clean_data.info())
train, test = train_test_split(clean_data, test_size=0.2)
train, test

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2994261 to 2371043
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   dayOfWeek  10000 non-null  int64
 1   hourOfDay  10000 non-null  int64
 2   clicked    10000 non-null  int64
dtypes: int64(3)
memory usage: 312.5 KB
None


(         dayOfWeek  hourOfDay  clicked
 801704           6         15        0
 99645            4          7        0
 897792           0          5        0
 487237           5          9        0
 866664           6         22        1
 ...            ...        ...      ...
 3592105          3         10        0
 1928364          3          8        0
 2197039          4          8        1
 1312829          1          9        0
 273906           4         17        0
 
 [8000 rows x 3 columns],
          dayOfWeek  hourOfDay  clicked
 1768653          2         18        0
 3312677          2          9        0
 2151268          4          6        0
 465442           5          8        0
 252135           4         15        1
 ...            ...        ...      ...
 2902361          0         15        0
 35638            4          5        0
 2262462          4         12        0
 1434366          1         18        1
 2111783          4          3        0
 
 [2000 row

<div dir=rtl'>
حالا به سراغ خود الگوریتم می‌رویم.
<br>

</div>

In [5]:

clsf.fit(train[['dayOfWeek','hourOfDay']], train[['clicked']])
real_y = test[['clicked']]
pred_y = clsf.predict(test[['dayOfWeek','hourOfDay']])
# (clsf.predict(newdata[0:2]), data[1:2].clicked)

In [ ]:
در پایان با استفاده از متریک‌های مختلف، یادگیری انجام شده را می‌سنجیم:

In [6]:
from sklearn import metrics
print("AUC : %.4f" %metrics.roc_auc_score(real_y, pred_y))
print("Cross-entropy loss : %.4f " %metrics.log_loss(real_y,pred_y))
print("F1 score : %.4f" %metrics.f1_score(real_y,pred_y,average="micro"))

AUC : 0.5000
Cross-entropy loss : 7.8403 
F1 score : 0.7730


In [ ]:
<div dir='rtl'>
اکنون برای استفاده بهتر از داده‌ها سعی میکنیم فیچر‌ها را ترتیبی کنیم.
<br>
نحوه کار این است که نمره هر کلاس را تعداد مشاهده‌ی آن در داده‌های کلیک شده می‌گذاریم
<div>

In [29]:
newdata=ps.DataFrame()

clickedsize = len(data[data.clicked==1])
def t(element):
    # print(val[element])
    try:
        if val[element] > 0:
            return val[element]/clickedsize
        else: return mean
    except Exception:
        return mean

for col in data.columns:
    if col in ['dayOfWeek', 'hourOfDay', 'clicked']:
        newdata[col] = data[col]
    else:
        val = data[data.clicked==1][col].value_counts()
        mean=np.mean(val)
        newdata[col] = data[col].map(t)


In [31]:
newtrain, newtest = train_test_split(newdata, test_size=0.2)

newtrain

clsf.fit(newtrain.drop(data.columns[[15]], axis=1), newtrain[['clicked']])
newreal_y = newtest[['clicked']]

newpred_y = clsf.predict(newtest.drop(data.columns[[15]], axis=1))

In [32]:
print("AUC : %.4f" %metrics.roc_auc_score(newreal_y, newpred_y))
print("Cross-entropy loss : %.4f " %metrics.log_loss(newreal_y,newpred_y))
print("F1 score : %.4f" %metrics.f1_score(newreal_y,newpred_y,average="micro"))

AUC : 0.9994
Cross-entropy loss : 0.0345 
F1 score : 0.9990
